# recsys data eda
all the current user movie user relation data analyse

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
%matplotlib inline

In [3]:
data_folder = '../dataset'

# movie and user csv data set
movie_comments_csv_path = '../dataset/DMSC.csv'
movie_csv_file_path = '../dataset/dataset1/movie.csv'
user_csv_file_path = '../dataset/dataset1/user.csv'

# user relation data set
edge_csv_file_path = '../dataset/dataset2/edges.csv'
node_csv_file_path = '../dataset/dataset2/nodes.csv'

# source json file path 
# json data just contains the movie information
movie_json_path = '../dataset/movie.json'
spider_json_path = '../dataset/spider.json'

## movie comments data
DMSC.csv

In [4]:
movie_comments = pd.read_csv(movie_comments_csv_path)
movie_comments.head()

,ID,Movie_Name_EN,Movie_Name_CN,Crawl_Date,Number,Username,Date,Star,Comment,Like
0,0,Avengers Age of Ultron,复仇者联盟2,2017-01-22,1,然潘,2015-05-13,3,连奥创都知道整容要去韩国。,2404
1,1,Avengers Age of Ultron,复仇者联盟2,2017-01-22,2,更深的白色,2015-04-24,2,非常失望，剧本完全敷衍了事，主线剧情没突破大家可以理解，可所有的人物都缺乏动机，正邪之间、...,1231
2,2,Avengers Age of Ultron,复仇者联盟2,2017-01-22,3,有意识的贱民,2015-04-26,2,2015年度最失望作品。以为面面俱到，实则画蛇添足；以为主题深刻，实则老调重弹；以为推陈出...,1052
3,3,Avengers Age of Ultron,复仇者联盟2,2017-01-22,4,不老的李大爷耶,2015-04-23,4,《铁人2》中勾引钢铁侠，《妇联1》中勾引鹰眼，《美队2》中勾引美国队长，在《妇联2》中终于...,1045
4,4,Avengers Age of Ultron,复仇者联盟2,2017-01-22,5,ZephyrO,2015-04-22,2,虽然从头打到尾，但是真的很无聊啊。,723


In [5]:
print(movie_comments.info())
print("some comments do not record the Username")
print(movie_comments.isna().any())
print("ID column equals index")
print(set(movie_comments['ID'].unique()) - set(movie_comments.index))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2125056 entries, 0 to 2125055
Data columns (total 10 columns):
ID               int64
Movie_Name_EN    object
Movie_Name_CN    object
Crawl_Date       object
Number           int64
Username         object
Date             object
Star             int64
Comment          object
Like             int64
dtypes: int64(4), object(6)
memory usage: 162.1+ MB
None
some comments do not record the Username
ID               False
Movie_Name_EN    False
Movie_Name_CN    False
Crawl_Date       False
Number           False
Username          True
Date             False
Star             False
Comment          False
Like             False
dtype: bool
ID column equals index
set()


In [6]:
res = movie_comments.groupby('Movie_Name_CN').size()
print("commented movies length: " + str(len(res)))
print(res.index.values)
print("all the moveis length in the comment data: " + str(len(movie_comments['Movie_Name_CN'].unique())))
# comments data just contains 28 movies.

commented movies length: 28
['七月与安生' '九层妖塔' '何以笙箫默' '你的名字' '十二生肖' '变形金刚4' '后会无期' '复仇者联盟' '复仇者联盟2'
 '夏洛特烦恼' '大圣归来' '大鱼海棠' '寻龙诀' '小时代1' '小时代3' '左耳' '栀子花开' '泰囧' '湄公河行动' '爱乐之城'
 '疯狂动物城' '美人鱼' '美国队长3' '西游伏妖篇' '西游降魔篇' '釜山行' '钢铁侠1' '长城']
all the moveis length in the comment data: 28


In [7]:
movie_df = pd.read_csv(movie_csv_file_path)
def func(item):
    if item.isnumeric() == True:
        return np.nan
    else:
        return item
print(len(movie_df['电影名'].unique()))
res = movie_df['电影名'].astype(str).apply(func).dropna()
print(res.head(10))
print('all the movies length in movie data: ' + str(len(res.unique())))

23034
0     我不是药神
1       老炮儿
2    唐人街探案2
3       驴得水
4      人在囧途
5       无人区
6      鬼子来了
7      无问西东
8      邪不压正
9    11度青春之
Name: 电影名, dtype: object
all the movies length in movie data: 22985


## user data

In [10]:
user_data = pd.read_csv(user_csv_file_path)
print(user_data.info())
print(user_data.head(3))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199813 entries, 0 to 199812
Data columns (total 6 columns):
评分      199813 non-null int64
用户名     199813 non-null object
评论时间    199813 non-null object
用户ID    199813 non-null int64
电影名     199813 non-null object
类型      199813 non-null object
dtypes: int64(2), object(4)
memory usage: 9.1+ MB
None
   评分     用户名                 评论时间  用户ID    电影名  类型
0   2      身似  2018-01-05 15:05:06     1   心雨花露  爱情
1   4  有意识的贱民  2018-01-05 15:05:06     3  战争的恐怖  战争
2   2    亿万露电  2018-01-05 15:05:06     4  豪勇七蛟龙  战争


In [50]:
print(user_data['评论时间'].dtype)
print(type(user_data['评论时间'][0]))

object
<class 'str'>


In [9]:
print(('-'*20 + '{}' + '-'*20).format("user information"))
print(user_data.info())
print(('-'*20 + '{}' + '-'*20).format("user information contains any null"))
print(user_data.isna().any())

--------------------user information--------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199813 entries, 0 to 199812
Data columns (total 6 columns):
评分      199813 non-null int64
用户名     199813 non-null object
评论时间    199813 non-null object
用户ID    199813 non-null int64
电影名     199813 non-null object
类型      199813 non-null object
dtypes: int64(2), object(4)
memory usage: 9.1+ MB
None
--------------------user information contains any null--------------------
评分      False
用户名     False
评论时间    False
用户ID    False
电影名     False
类型      False
dtype: bool


In [11]:
print("total length of movie {}".format(len(user_data['电影名'])))
print("unique movie length {}".format(len(user_data['电影名'].unique())))
# have duplicated movies.

total length of movie 199813
unique movie length 23031


In [66]:
# analyse the rating range
print(('-'*20 + '{}' + '-'*20).format("all the rating values"))
print(user_data['评分'].unique())
print("max rating {} and min rating {} ".format(max(user_data['评分']), min(user_data['评分'])))
print("the rating user name length is {} ".format(len(user_data['用户名'].unique())))
print("the rating user id length is {} ".format(len(user_data['用户ID'].unique())))

print("the total movie length is {} ".format(len(user_data['电影名'].unique())))
# same with the dataframe size so there is no duplicated user, movie pair
print("drop duplicates date length is {} ".format(len(user_data.drop_duplicates(['用户名', '电影名']))))


print("the total movie types length is {}".format(len(user_data['类型'].unique())))
print(('-'*20 + '{}' + '-'*20).format("total movie types"))
print(user_data['类型'].unique())
print(('-'*20 + '{}' + '-'*20).format("rating time range"))
# use print will show object not use print will show dtype('o')
print(user_data['评论时间'].dtype)
# str type
print(type(user_data['评论时间'][0]))
# convert datetime str to pd.Timestamp type
user_data['评论时间'] = pd.to_datetime(user_data['评论时间'])
print(user_data['评论时间'].dtype)
print(type(user_data['评论时间'][0]))
print("lateast comment time {} and oldest comment time {}".format(user_data['评论时间'].max(), user_data['评论时间'].min()))

--------------------all the rating values--------------------
[ 2  4  8 10  6]
max rating 10 and min rating 2 
the rating user name length is 13532 
the rating user id length is 13545 
the total movie length is 23031 
drop duplicates date length is 199813 
the total movie types length is 21
--------------------total movie types--------------------
['爱情' '战争' '犯罪' '同性' '喜剧' '冒险' '歌舞' '情色' '剧情' '奇幻' '音乐' '武侠' '恐怖' '传记'
 '悬疑' '历史' '惊悚' '动作' '科幻' '西部' '灾难']
--------------------rating time range--------------------
datetime64[ns]
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
datetime64[ns]
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
lateast comment time 2018-02-07 22:02:57 and oldest comment time 2018-01-05 15:05:06


## movie data

In [39]:
movie_data = pd.read_csv(movie_csv_file_path)

In [41]:
print(('-'*20 + '{}' + '-'*20).format("movie data information"))
print(movie_data.info())
print(('-'*20 + '{}' + '-'*20).format("movie data contains null data"))
print(movie_data.isna().any())

--------------------movie data information--------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93160 entries, 0 to 93159
Data columns (total 7 columns):
类型     93160 non-null object
主演     93160 non-null object
地区     93160 non-null object
导演     93160 non-null object
特色     93160 non-null object
评分     93160 non-null float64
电影名    93160 non-null object
dtypes: float64(1), object(6)
memory usage: 5.0+ MB
None
--------------------movie data contains null data--------------------
类型     False
主演     False
地区     False
导演     False
特色     False
评分     False
电影名    False
dtype: bool


In [63]:
print(movie_data['类型'].head(3))
print(('-'*20 + '{}' + '-'*20).format("unique movie type"))
# 21 types same with the user data
print(movie_data['类型'].unique())
# | split data
print(movie_data['主演'].head(3))

print(movie_data['地区'].head(3))

print(movie_data['导演'].head(3))

print(movie_data['特色'].head(3))

# 9 customised tags about movies
print(movie_data['特色'].unique())

# just 23034 unique movies
print("unique movie length {} " .format(len(movie_data['电影名'].unique())))

0    剧情
1    剧情
2    剧情
Name: 类型, dtype: object
--------------------unique movie type--------------------
['剧情' '喜剧' '动作' '爱情' '科幻' '悬疑' '惊悚' '恐怖' '犯罪' '同性' '冒险' '音乐' '歌舞' '传记'
 '历史' '西部' '战争' '情色' '武侠' '灾难' '奇幻']
0      徐峥|王传君|周一围|谭卓|章宇
1     冯小刚|许晴|张涵予|刘桦|李易峰
2    王宝强|刘昊然|肖央|刘承羽|尚语贤
Name: 主演, dtype: object
0    中国大陆
1    中国大陆
2    中国大陆
Name: 地区, dtype: object
0    文牧野
1     管虎
2    陈思诚
Name: 导演, dtype: object
0    经典
1    经典
2    经典
Name: 特色, dtype: object
['经典' '青春' '文艺' '搞笑' '励志' '魔幻' '感人' '女性' '黑帮']
movie length 23034 


In [70]:
# expand the column
movie_data['主演'].drop_duplicates().str.split('|', expand=True).stack().reset_index(level=1, drop=True)

0                                 徐峥
0                                王传君
0                                周一围
0                                 谭卓
0                                 章宇
1                                冯小刚
1                                 许晴
1                                张涵予
1                                 刘桦
1                                李易峰
2                                王宝强
2                                刘昊然
2                                 肖央
2                                刘承羽
2                                尚语贤
3                                任素汐
3                                 大力
3                                刘帅良
3                                裴魁山
3                                阿如那
4                                 徐峥
4                                王宝强
4                                 李曼
4                                李小璐
4                                左小青
5                                 徐峥
5                                 黄渤
5

# CF recall process

## deal with id
in spark als algo userId and movieId should be integer
so should maintain a look up table to save the integer id and source id 
of users and movies.